# Agent Loan Decisioning w CSV files

In [165]:
import sys
import datetime
from datetime import date,timedelta
import numpy as np
import pandas as pd
# import camelot
import matplotlib.pyplot as plt
import logging

In [166]:
pd.set_option('display.max_columns',None)

In [167]:
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logging.info('Code Begin')

2023-08-07 01:01:59,499 - Code Begin



Qualifying Matrix

1.	Agent Length of Service - 3 months

2.	Agent Frequency of use/activity - >= 300 transactions(count) in total

3.	Agent Frequency of rebalance - >= 50 inbound(count) in total

4.	Median Cash-In for last 90 days - avg. txn amnt - ghs800 

5.	Mode Cash-In for last 90 days - lowest amnt cash in - ghs400 

6.	Median Cash-Out for last 90 days - avg. txn amnt - ghs1000

7.	Mode Cash-Out for last 90 days - lowest amnt cash out - ghs500 

In [168]:
# Windows

# table_fin = pd.read_csv(r'C:\Users\Chakdahah\ReginaOdoi.csv')
# table_fin = pd.read_csv(r'C:\Users\Chakdahah\Downloads\newmetrics\StrippedMomotoCSVAntwi.csv')

# MAC
# table_fin = pd.read_csv('/Users/otema/benjamin_adjei_statement.csv')
table_fin = pd.read_csv('/Users/otema/ReginaOdoi.csv')

# table_fin = pd.read_csv('/Users/otema/StrippedMomotoCSVventuremaxnew.csv') 

table_fin.head() 

,Id,External Transaction Id,Date,Status,Type,Provider Category,To Message,From,From Name,To,To Name,Initiated By,On Behalf Of,Approved By,Amount,Currency,Fee,Currency.1,Discount,Currency.2,Promotion,Currency.3,Coupon,Currency.4,Balance,Currency.5,Information
0,26713190285,NaN,7/31/2023 8:04,Successful,Transfer,NaN,1,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597431338/MSISDN,REGINA ODOI,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,-600,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2,GHS,NaN
1,26712899802,230731MMB7671313,7/31/2023 7:54,Successful,External payment,NaN,NaN,FRI:233244367610/MSISDN,REGINA ODOI,FRI:1055753063311@fblpush/SP,FBL,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,-400,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,602,GHS,NaN
2,26712875501,NaN,7/31/2023 7:53,Successful,Transfer,NaN,1,FRI:233597431338/MSISDN,REGINA ODOI,FRI:233244367610/MSISDN,REGINA ODOI,ID:233597431338/MSISDN,ID:233597431338/MSISDN,NaN,400,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1002,GHS,NaN
3,26712810998,NaN,7/31/2023 7:51,Successful,Transfer,NaN,1,FRI:233549496456/MSISDN,EMMANUEL ABOAGYE,FRI:233244367610/MSISDN,REGINA ODOI,ID:233549496456/MSISDN,ID:233549496456/MSISDN,NaN,600,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,602,GHS,NaN
4,26712175971,NaN,7/31/2023 7:25,Successful,Transfer,NaN,1,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597431338/MSISDN,REGINA ODOI,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,-370,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2,GHS,NaN


In [169]:

if 'Unnamed: 0' in table_fin.columns:
    table_fin = table_fin.iloc[:,1:]

In [170]:
table_fin.head()

,Id,External Transaction Id,Date,Status,Type,Provider Category,To Message,From,From Name,To,To Name,Initiated By,On Behalf Of,Approved By,Amount,Currency,Fee,Currency.1,Discount,Currency.2,Promotion,Currency.3,Coupon,Currency.4,Balance,Currency.5,Information
0,26713190285,NaN,7/31/2023 8:04,Successful,Transfer,NaN,1,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597431338/MSISDN,REGINA ODOI,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,-600,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2,GHS,NaN
1,26712899802,230731MMB7671313,7/31/2023 7:54,Successful,External payment,NaN,NaN,FRI:233244367610/MSISDN,REGINA ODOI,FRI:1055753063311@fblpush/SP,FBL,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,-400,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,602,GHS,NaN
2,26712875501,NaN,7/31/2023 7:53,Successful,Transfer,NaN,1,FRI:233597431338/MSISDN,REGINA ODOI,FRI:233244367610/MSISDN,REGINA ODOI,ID:233597431338/MSISDN,ID:233597431338/MSISDN,NaN,400,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1002,GHS,NaN
3,26712810998,NaN,7/31/2023 7:51,Successful,Transfer,NaN,1,FRI:233549496456/MSISDN,EMMANUEL ABOAGYE,FRI:233244367610/MSISDN,REGINA ODOI,ID:233549496456/MSISDN,ID:233549496456/MSISDN,NaN,600,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,602,GHS,NaN
4,26712175971,NaN,7/31/2023 7:25,Successful,Transfer,NaN,1,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597431338/MSISDN,REGINA ODOI,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,-370,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2,GHS,NaN


In [171]:
# Assign tables into momo data variable

momo_data = table_fin

In [172]:
#Reading data from csv file

# momo_data = pd.read_csv(r'\Users\Chakdahah\Cleaned_Momo.csv', skipinitialspace = True)

# Clean whitespaces in colunn name and entire dataframe

momo_data =momo_data.rename(columns=lambda x: x.strip())
momo_data =momo_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

momo_data.head()

,Id,External Transaction Id,Date,Status,Type,Provider Category,To Message,From,From Name,To,To Name,Initiated By,On Behalf Of,Approved By,Amount,Currency,Fee,Currency.1,Discount,Currency.2,Promotion,Currency.3,Coupon,Currency.4,Balance,Currency.5,Information
0,26713190285,NaN,7/31/2023 8:04,Successful,Transfer,NaN,1,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597431338/MSISDN,REGINA ODOI,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,-600,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2,GHS,NaN
1,26712899802,230731MMB7671313,7/31/2023 7:54,Successful,External payment,NaN,NaN,FRI:233244367610/MSISDN,REGINA ODOI,FRI:1055753063311@fblpush/SP,FBL,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,-400,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,602,GHS,NaN
2,26712875501,NaN,7/31/2023 7:53,Successful,Transfer,NaN,1,FRI:233597431338/MSISDN,REGINA ODOI,FRI:233244367610/MSISDN,REGINA ODOI,ID:233597431338/MSISDN,ID:233597431338/MSISDN,NaN,400,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1002,GHS,NaN
3,26712810998,NaN,7/31/2023 7:51,Successful,Transfer,NaN,1,FRI:233549496456/MSISDN,EMMANUEL ABOAGYE,FRI:233244367610/MSISDN,REGINA ODOI,ID:233549496456/MSISDN,ID:233549496456/MSISDN,NaN,600,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,602,GHS,NaN
4,26712175971,NaN,7/31/2023 7:25,Successful,Transfer,NaN,1,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597431338/MSISDN,REGINA ODOI,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,-370,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2,GHS,NaN


In [173]:

# remove all unwanted characters in column names

momo_data.rename(columns=lambda s: s.replace(".", ""), inplace=True)

# replace whitespaces with underscore
momo_data.rename(columns=lambda s: s.replace(" ", "_"), inplace=True)

In [174]:

if 'Id' in momo_data:
    momo_data.rename(
    columns = {'From':"FROM_NO", "From_Name":"FROM_NAME","Type":"TRANS_TYPE",
              "Amount":"AMOUNT","Initiated_By":"OVA","Date":"TRANSACTION_DATE"},
                inplace = True)    
    


In [175]:
momo_data["AMOUNT"] = momo_data["AMOUNT"].astype(str)

In [176]:
# remove negative sign in amount column

momo_data["AMOUNT"] = momo_data["AMOUNT"].str.replace("-","")

In [177]:
momo_data["TRANS_TYPE"] = momo_data["TRANS_TYPE"].str.upper()
momo_data["TRANS_TYPE"] = momo_data["TRANS_TYPE"].str.replace(" ","_")

In [178]:
momo_data.head()

,Id,External_Transaction_Id,TRANSACTION_DATE,Status,TRANS_TYPE,Provider_Category,To_Message,FROM_NO,FROM_NAME,To,To_Name,OVA,On_Behalf_Of,Approved_By,AMOUNT,Currency,Fee,Currency1,Discount,Currency2,Promotion,Currency3,Coupon,Currency4,Balance,Currency5,Information
0,26713190285,NaN,7/31/2023 8:04,Successful,TRANSFER,NaN,1,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597431338/MSISDN,REGINA ODOI,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,600.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2,GHS,NaN
1,26712899802,230731MMB7671313,7/31/2023 7:54,Successful,EXTERNAL_PAYMENT,NaN,NaN,FRI:233244367610/MSISDN,REGINA ODOI,FRI:1055753063311@fblpush/SP,FBL,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,400.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,602,GHS,NaN
2,26712875501,NaN,7/31/2023 7:53,Successful,TRANSFER,NaN,1,FRI:233597431338/MSISDN,REGINA ODOI,FRI:233244367610/MSISDN,REGINA ODOI,ID:233597431338/MSISDN,ID:233597431338/MSISDN,NaN,400.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1002,GHS,NaN
3,26712810998,NaN,7/31/2023 7:51,Successful,TRANSFER,NaN,1,FRI:233549496456/MSISDN,EMMANUEL ABOAGYE,FRI:233244367610/MSISDN,REGINA ODOI,ID:233549496456/MSISDN,ID:233549496456/MSISDN,NaN,600.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,602,GHS,NaN
4,26712175971,NaN,7/31/2023 7:25,Successful,TRANSFER,NaN,1,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597431338/MSISDN,REGINA ODOI,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,370.0,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2,GHS,NaN


In [179]:
# cast appropriate data types
momo_data['AMOUNT'] = pd.to_numeric(momo_data['AMOUNT'],errors='coerce')
# momo_data['BAL_BEFORE'] = pd.to_numeric(momo_data['BAL_BEFORE'],errors='coerce')
# momo_data['BAL_AFTER'] = pd.to_numeric(momo_data['BAL_AFTER'],errors='coerce')
# momo_data['F_ID'] = pd.to_numeric(momo_data['F_ID'],errors='coerce')


In [180]:
momo_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5662 entries, 0 to 5661
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       5662 non-null   int64  
 1   External_Transaction_Id  94 non-null     object 
 2   TRANSACTION_DATE         5662 non-null   object 
 3   Status                   5662 non-null   object 
 4   TRANS_TYPE               5662 non-null   object 
 5   Provider_Category        0 non-null      float64
 6   To_Message               5547 non-null   object 
 7   FROM_NO                  5662 non-null   object 
 8   FROM_NAME                5662 non-null   object 
 9   To                       5662 non-null   object 
 10  To_Name                  5662 non-null   object 
 11  OVA                      5662 non-null   object 
 12  On_Behalf_Of             5662 non-null   object 
 13  Approved_By              1 non-null      object 
 14  AMOUNT                  

In [181]:
#Suppressing any scientific notation in entire csv file.

pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [182]:
if 'TRANSACTION_TIME' in momo_data:
    print("in it")
    
else:
    m =momo_data['TRANSACTION_DATE'].str.split(' ', expand=True)
    momo_data['TRANSACTION_DATE'] = m[0]
    momo_data['TRANSACTION_TIME'] = m[1]

    first_column = momo_data.pop('TRANSACTION_TIME')
  
    # insert column using insert(position,column_name,first_column) function
    momo_data.insert(1, 'TRANSACTION_TIME', first_column)
 

In [183]:
momo_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5662 entries, 0 to 5661
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       5662 non-null   int64  
 1   TRANSACTION_TIME         5662 non-null   object 
 2   External_Transaction_Id  94 non-null     object 
 3   TRANSACTION_DATE         5662 non-null   object 
 4   Status                   5662 non-null   object 
 5   TRANS_TYPE               5662 non-null   object 
 6   Provider_Category        0 non-null      float64
 7   To_Message               5547 non-null   object 
 8   FROM_NO                  5662 non-null   object 
 9   FROM_NAME                5662 non-null   object 
 10  To                       5662 non-null   object 
 11  To_Name                  5662 non-null   object 
 12  OVA                      5662 non-null   object 
 13  On_Behalf_Of             5662 non-null   object 
 14  Approved_By             

In [184]:
momo_data["FROM_NO"] = momo_data["FROM_NO"].astype(str)

In [185]:
if momo_data['FROM_NO'].str.contains(":",case = False).any():
    
    h = momo_data['FROM_NO'].iloc[0]
    # here ; and / are our two markers 
    # in which string can be found. 
    num = h.split(':')[1].split('/')[0]
    print(num)
    
    ovah = momo_data['OVA'].iloc[0]
    # here ; and / are our two markers 
    # in which string can be found. 
    ova = ovah.split(':')[1].split('/')[0]
    print(ova)
    
    if ova == num:
        name = momo_data['FROM_NAME'].iloc[0]
    
else:
    num = momo_data.loc[momo_data['TRANS_TYPE']=='DEBIT','FROM_NO'].iloc[0] #Median Cash-In for last 90days
    num
    
    name =  momo_data.loc[momo_data['TRANS_TYPE']=='DEBIT','FROM_NAME'].iloc[0]
    name
    
    

233244367610
233244367610


In [186]:
momo_data['TRANSACTION_DATE'] = pd.to_datetime(momo_data['TRANSACTION_DATE'])

In [187]:

momo_data['TRANSACTION_DATE'] = momo_data['TRANSACTION_DATE'].dt.strftime('%b')
momo_data

,Id,TRANSACTION_TIME,External_Transaction_Id,TRANSACTION_DATE,Status,TRANS_TYPE,Provider_Category,To_Message,FROM_NO,FROM_NAME,To,To_Name,OVA,On_Behalf_Of,Approved_By,AMOUNT,Currency,Fee,Currency1,Discount,Currency2,Promotion,Currency3,Coupon,Currency4,Balance,Currency5,Information
0,26713190285,8:04,NaN,Jul,Successful,TRANSFER,NaN,1,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597431338/MSISDN,REGINA ODOI,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,600,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2,GHS,NaN
1,26712899802,7:54,230731MMB7671313,Jul,Successful,EXTERNAL_PAYMENT,NaN,NaN,FRI:233244367610/MSISDN,REGINA ODOI,FRI:1055753063311@fblpush/SP,FBL,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,400,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,602,GHS,NaN
2,26712875501,7:53,NaN,Jul,Successful,TRANSFER,NaN,1,FRI:233597431338/MSISDN,REGINA ODOI,FRI:233244367610/MSISDN,REGINA ODOI,ID:233597431338/MSISDN,ID:233597431338/MSISDN,NaN,400,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,1002,GHS,NaN
3,26712810998,7:51,NaN,Jul,Successful,TRANSFER,NaN,1,FRI:233549496456/MSISDN,EMMANUEL ABOAGYE,FRI:233244367610/MSISDN,REGINA ODOI,ID:233549496456/MSISDN,ID:233549496456/MSISDN,NaN,600,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,602,GHS,NaN
4,26712175971,7:25,NaN,Jul,Successful,TRANSFER,NaN,1,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597431338/MSISDN,REGINA ODOI,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,370,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,2,GHS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5657,24101464102,07:30,NaN,May,Successful,CASH_IN,NaN,Cashin To-233544951082,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233544951082/MSISDN,Osei Kwaku,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,70,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,4885,GHS,NaN
5658,24101319132,07:24,NaN,May,Successful,CASH_OUT,NaN,NationalId--,FRI:233244422160/MSISDN,Peter Douglas Laryea Tetteh,FRI:233244367610/MSISDN,REGINA ODOI,ID:233244422160/MSISDN,ID:233244422160/MSISDN,NaN,990,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,4955,GHS,NaN
5659,24101221205,07:20,NaN,May,Successful,CASH_IN,NaN,Cashin To-233597846534,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233597846534/MSISDN,AKUA KWAKYEWAA,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,81,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,3965,GHS,NaN
5660,24100460893,06:41,NaN,May,Successful,CASH_IN,NaN,Cashin To-233552393401,FRI:233244367610/MSISDN,REGINA ODOI,FRI:233552393401/MSISDN,Firdaws Tambili,ID:233244367610/MSISDN,ID:233244367610/MSISDN,NaN,60,GHS,0,GHS,NaN,NaN,NaN,NaN,NaN,NaN,4046,GHS,NaN


In [188]:
# Set empty list and dict
masterData = []
userData= {}

# Calculation of scoring parameters

In [189]:

# # considering cashin, cashout for activity 

R = momo_data.loc[(momo_data['TRANS_TYPE'] == 'CASH_IN') | (momo_data['TRANS_TYPE'] == 'CASH_OUT')]
rowsie = R.count()[0]
userData["freq_of_active"] = rowsie


# do transfer where from_no =! account holder (num)

MOMO=momo_data.loc[(momo_data['TRANS_TYPE'] == 'TRANSFER') & (momo_data['FROM_NO'] != num)] # frequency of rebalance
MOMOR = MOMO.count()[0]
userData["freq_of_repl"] = MOMOR


meadnie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].median() #Median Cash-In for last 90days
# userData["mead_cash_in"] = meadnie
# userData["mead_cash_in"]
userData["median_cash_in_calc"] = meadnie*4


modnie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].mode() # Mode Cash-In for last 90 days
userData["mode_cash_in_calc"] = modnie*4
userData["mode_cash_in_calc"] = max(userData["mode_cash_in_calc"]) #for converting from bool to int for processing


meadniout = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].median() # Median Cash-Out for last 90 days
# userData["mead_cash_out"] = meadniout
# userData["mead_cash_out"]
userData["median_cash_out_calc"] = meadniout*4


modno = momo_data.loc[momo_data['TRANS_TYPE'] == 'CASH_OUT', 'AMOUNT'].mode()  # Mode Cash-Out for last 90 days
userData["mode_cash_out_calc"] = modno * 4 
userData["mode_cash_out_calc"] = max(userData["mode_cash_out_calc"])



# Scoring Criteria Calculation

In [190]:
user_copy = userData.copy()
masterData.append(user_copy)

In [191]:
### scoring

inelligible = []
elligible = []
score_table = []
scoring_table = {}
combined = []

In [192]:
scoring_table["Agent Number"] = num
scoring_table["Agent Name"] = name

In [193]:
mi =momo_data['TRANSACTION_DATE']

In [194]:

# # Agent Length of Service - 3 months

for i in mi:
    u = mi.str.extract('([a-zA-Z ]+)', expand=False).str.strip()
    monthcount = u.drop_duplicates()
print(monthcount)

if len(monthcount) >= 3:
    scoring_table["Length of Service"] = 100
    Credit_score = 100
else:
    scoring_table["Length of Service"] = 0
    Credit_score = 0

for index in range(len(masterData)):

    # # Frequency of use  >= 300 transactions(count) in total

    if masterData[index]["freq_of_active"] >= 300 and masterData[index]["freq_of_active"] <= 600:
        user_copy = masterData[index]
#         print(user_copy)
        Credit_score += 50
        scoring_table["Frequency of Activity"] = 50
    elif masterData[index]["freq_of_active"] > 600:
        elig_copy = masterData[index]
        Credit_score += 100
        scoring_table["Frequency of Activity"] = 100
    else:
        scoring_table["Frequency of Activity"] = 0

# Agent Frequency of rebalance - >= 50 inbound(count) in total

    if masterData[index]["freq_of_repl"] >= 50 and masterData[index]["freq_of_repl"] <= 100:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Frequency of Rebalance"] = 50
    elif masterData[index]["freq_of_repl"] > 100:
        elig_copy = masterData[index]
        Credit_score += 100
        scoring_table["Frequency of Rebalance"] = 100
    else:
        scoring_table["Frequency of Rebalance"] = 0

        # Median Cash-In for last 90 days - avg. txn amnt - ghs800 

    if masterData[index]["median_cash_in_calc"] >= 800:
        user_copy = masterData[index]
        Credit_score += 50
        scoring_table["Median Cash In"] = 50
    else:
        Credit_score += 0
        scoring_table["Median Cash In"] = 0

        # Mode Cash-In for last 90 days - lowest amnt cash in - ghs400 

    if masterData[index]["mode_cash_in_calc"] >= 400:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Mode Cash In"] = 50

    else:
        elig_copy = masterData[index]
        Credit_score += 0
        scoring_table["Mode Cash In"] = 0

    # Median Cash-Out for last 90 days - avg. txn amnt - ghs1000

    if masterData[index]["median_cash_out_calc"] > 1000:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Median Cash Out"] = 50
    else:
        elig_copy = masterData[index]
        Credit_score += 0
        scoring_table["Median Cash Out"] = 0

#     Mode Cash-Out for last 90 days - lowest amnt cash out - ghs500 

    if masterData[index]["mode_cash_out_calc"] >= 500:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Mode Cash Out"] = 50
    else:
        elig_copy = masterData[index]
        Credit_score += 0   
        scoring_table["Mode Cash Out"] = 0

    

#         print(elig_copy)
scoring_table["Credit Score"] = Credit_score
score_table.append(scoring_table)        

0       Jul
1739    Jun
3594    May
Name: TRANSACTION_DATE, dtype: object


In [195]:
elligible.append(elig_copy)

# Applying scores to Merchants

In [196]:

if Credit_score < 200:
    print("You don't qualify for a loan")
    scoring_table["Eligibility"] = 0

elif Credit_score >= 200 and Credit_score <= 250:
    print("You qualify for a GHS 1000 loan")
    scoring_table["Eligibility"] = 1000

elif Credit_score >= 251 and Credit_score <= 300:
    print("You qualify for a GHS 2000 loan")
    scoring_table["Eligibility"] = 2000

elif Credit_score >= 301 and Credit_score <= 350:
    print("You qualify for a GHS 2500 loan")
    scoring_table["Eligibility"] = 2500 

elif Credit_score >= 351 and Credit_score <= 400:
    print("You qualify for a GHS 3000 loan")
    scoring_table["Eligibility"] = 3000

elif Credit_score >= 401 and Credit_score <= 450:
    print("You qualify for a GHS 4000 loan")
    scoring_table["Eligibility"] = 4000

elif Credit_score >= 451 and Credit_score <= 500:
    print("You qualify for a GHS 5000 loan")
    scoring_table["Eligibility"] = 5000


print("Credit Score: ", Credit_score)


You qualify for a GHS 2500 loan
Credit Score:  350


# Combining Eligibility and Score Tables

In [197]:
scoring_table.update(elig_copy)
comb = scoring_table
comb


{'Agent Number': '233244367610',
 'Agent Name': 'REGINA ODOI',
 'Length of Service': 100,
 'Frequency of Activity': 100,
 'Frequency of Rebalance': 100,
 'Median Cash In': 0,
 'Mode Cash In': 50,
 'Median Cash Out': 0,
 'Mode Cash Out': 0,
 'Credit Score': 350,
 'Eligibility': 2500,
 'freq_of_active': 4963,
 'freq_of_repl': 611,
 'median_cash_in_calc': 400.0,
 'mode_cash_in_calc': 400.0,
 'median_cash_out_calc': 600.0,
 'mode_cash_out_calc': 400.0}

In [198]:
# comb = scoring_table | elig_copy
# comb


In [199]:
combined.append(comb)

In [200]:
print("Elligible", elligible)

Elligible [{'freq_of_active': 4963, 'freq_of_repl': 611, 'median_cash_in_calc': 400.0, 'mode_cash_in_calc': 400.0, 'median_cash_out_calc': 600.0, 'mode_cash_out_calc': 400.0}]


In [201]:
Credit_score

350

# Converting and saving Combined Table

In [202]:
combined_table = pd.DataFrame.from_dict(combined)
combined_table

,Agent Number,Agent Name,Length of Service,Frequency of Activity,Frequency of Rebalance,Median Cash In,Mode Cash In,Median Cash Out,Mode Cash Out,Credit Score,Eligibility,freq_of_active,freq_of_repl,median_cash_in_calc,mode_cash_in_calc,median_cash_out_calc,mode_cash_out_calc
0,233244367610,REGINA ODOI,100,100,100,0,50,0,0,350,2500,4963,611,400,400,600,400


In [203]:

combined_table.to_csv('/users/otema/combtabletrials.csv')

In [204]:
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logging.info('End of Code')

2023-08-07 01:02:50,165 - End of Code


# Score and risk grade table 

In [205]:
details = {
    'Risk Ranking': ['RR-1', 'RR-2A', 'RR-2B', 'RR-3A', 'RR-3B', 'RR-4'],
    'Minimum Score': [200, 251, 301, 351, 401, 451],
    'Maximum Score': [250, 300, 350, 400, 450, 500],
    'Qualifying Amount(GHS)':[1000, 2000, 2500, 3000, 4000, 5000, ]
}
  
# creating a Dataframe object 
df = pd.DataFrame(details)
df

,Risk Ranking,Minimum Score,Maximum Score,Qualifying Amount(GHS)
0,RR-1,200,250,1000
1,RR-2A,251,300,2000
2,RR-2B,301,350,2500
3,RR-3A,351,400,3000
4,RR-3B,401,450,4000
5,RR-4,451,500,5000
